In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from ieee_lgb import eval_train
from kaggle_utils import reduce_mem_usage, move_feature

In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

paths_train = glob('../feature/raw_use/*_train.gz')
paths_test = glob('../feature/raw_use/*_test.gz')

paths_train = [path for path in paths_train 
               if path.count('Fraud') 
               or path.count(COLUMN_ID)
               or path.count('D')
               or path.count('C')
               or path.count('V')
               or path.count('amt')
               or path.count('card')
               or path.count('addr')
#                or path.count('Reg')
               or path.count('P_email')
               or path.count('R_email')
               or path.count('M')
               or path.count('Product')
              ]
paths_test  = [path for path in paths_test  
               if path.count('Fraud') 
               or path.count(COLUMN_ID)
               or path.count('D')
               or path.count('C')
               or path.count('V')
               or path.count('amt')
               or path.count('card')
               or path.count('addr')
#                or path.count('Reg')
               or path.count('P_email')
               or path.count('R_email')
               or path.count('M')
               or path.count('Product')
              ]

df_train = reduce_mem_usage( parallel_load_data(paths_train) )
df_test  = reduce_mem_usage( parallel_load_data(paths_test) )

Memory usage of dataframe is 297.92 MB
Memory usage after optimization is: 306.93 MB
Decreased by -3.0%
Memory usage of dataframe is 478.39 MB
Memory usage after optimization is: 541.20 MB
Decreased by -13.1%


Process ForkPoolWorker-127:
Process ForkPoolWorker-118:
Process ForkPoolWorker-103:
Process ForkPoolWorker-102:
Process ForkPoolWorker-75:
Process ForkPoolWorker-61:
Process ForkPoolWorker-100:
Process ForkPoolWorker-106:
Process ForkPoolWorker-109:
Process ForkPoolWorker-105:
Process ForkPoolWorker-50:
Process ForkPoolWorker-111:
Process ForkPoolWorker-115:
Process ForkPoolWorker-121:
Process ForkPoolWorker-40:
Process ForkPoolWorker-99:
Process ForkPoolWorker-76:
Process ForkPoolWorker-98:
Process ForkPoolWorker-24:
Process ForkPoolWorker-30:
Process ForkPoolWorker-110:
Process ForkPoolWorker-113:
Process ForkPoolWorker-48:
Process ForkPoolWorker-65:
Process ForkPoolWorker-22:
Process ForkPoolWorker-47:
Process ForkPoolWorker-31:
Process ForkPoolWorker-119:
Process ForkPoolWorker-124:
Process ForkPoolWorker-84:
Process ForkPoolWorker-85:
Process ForkPoolWorker-117:
Process ForkPoolWorker-101:
Process ForkPoolWorker-68:
Process ForkPoolWorker-112:
Process ForkPoolWorker-122:
Process F

In [3]:
Y = df_train[COLUMN_TARGET]
df_train.drop(COLUMN_TARGET, axis=1, inplace=True)

In [11]:
same_user_path = '../output/same_user_pattern/20190901_user_ids_share.csv'
model_type = "lgb"
params = {
    'n_jobs': 60,
    'seed': 1208,
    'n_splits': 5,
    'metric': 'auc',
    'model_type': model_type,
    'objective': 'binary',
    'fold': ['stratified', 'group'][1],

    'num_leaves': 2**6-1,
    'max_depth': -1,
    'subsample': 1.0,
    'subsample_freq': 1,
    'colsample_bytree' : 0.25,
    'lambda_l1' : 0.1,
    'lambda_l2' : 1.0,
    'learning_rate' : 0.1,
}
df_feim, adv_df_feim = eval_train(
    df_train,
    Y,
    df_test,
    same_user_path,
    model_type,
    params,
    is_adv=True,
    is_corr=False,
    is_viz=True,
)

  1%|          | 3/248 [00:00<00:09, 26.65it/s]

* Check Unique Feature.


100%|██████████| 248/248 [00:02<00:00, 85.89it/s]


  * 1 No Info Features: ['is_train']
  * 1feature unique drop and move trush
['is_train']


FileNotFoundError: [Errno 2] No such file or directory: '/home/yryrgogo/github/ieee-fraud/feature/raw_use/raw__is_train_train.gz'